# User Guide for CRUD SQLite operations in Python

## Connection to database

In [6]:
# without context manager
import sqlite3

#create the file path
file = "my_data.db" # "http://somewhere.com/database"

#connect to this path
conn = sqlite3.connect(file)

#create a cursor object to execute all operations
c = conn.cursor()

#After operation have been done connect object must commit the changes
#conn.commit()

#At the end always close your file
#conn.close()

In [105]:
#with context manager
from sqlite3 import Error
con = sqlite3.connect(":memory:")
con.execute('CREATE TABLE employee (id integer primary key, firstname varchar(20), email varchar(30) unique)')

# Successful con.commity is automatically called
with con:
    con.execute("INSERT INTO employee(firstname, email) VALUES (?,?)", ('Briann','briann@mycompany.com'))
                
    try:
   
        con.execute("INSERT INTO employee(firstname, email) VALUES (?,?)", ('Briann','briann@mycompany.com'))
       
    except Error as err:
        print("Something went wrong:  {}".format(err))
   

    c = con.cursor()
    c.execute("SELECT * FROM employee")
    print(c.fetchall())



    
               
                
                
                
                
    

Something went wrong:  UNIQUE constraint failed: employee.email
[(1, 'Briann', 'briann@mycompany.com')]


## CRUD Create Read Update Delete


## Create Tables

In [62]:
#First Create table
with conn:
    c = conn.cursor()
    c.execute('''CREATE TABLE stocks 
                    (date text, symbol text, qty real, price real)''')

#Insert values
c.execute("INSERT INTO stocks VALUES ('2019-01-05', 'APPL', 50, 300.35)")

#commit the changes
conn.commit()

#close the connection
conn.close()


### SQL INSERT INTO Statement

In [64]:
try:
    conn = sqlite3.connect("yahoostocks.db")
    with conn:
        c = conn.cursor()
        query = 'INSERT INTO stocks_prices VALUES("new_stock", 2019.14, "2019-04-01 13:05:59")'
        c.execute(query)
        print("Inserted into data base")
except Error as err:
    print("Error: {}".format(err))


Inserted into data base


In [65]:
try:
    conn = sqlite3.connect("yahoostocks.db")
    with conn:
        c = conn.cursor()
        query = "INSERT INTO stocks_prices (stock_name, price) VALUES('Appl', 444.44 )"
        c.execute(query)
        print("Inserted into data base")
except Error as err:
    print("Error: {}".format(err))



Inserted into data base


## Read or Querying the Tables

In [66]:
# CONNECT TO YAHOOSTOCK DATABASE TABLE stocks_prices
# HAS COLUMNS stock_name, price, time1
try:
    con = sqlite3.connect("yahoostocks.db")
except Error as err:
    print("Failed to connect to database: {}".format(err))

In [67]:
stock = "Dow"
#for secure format for querying to avoid SQL injection
try:
    with con:
        c = con.cursor()
        c.execute('SELECT * FROM stocks_prices WHERE stock_name = ?',(stock,) )
        print(c.fetchone())
        for row in c:
            print(row)
except Error as err:
    print("Failed to get query: {}".format(err))
    

('Dow', '26,183.44', '2019-04-01 12:44:50')
('Dow', '26,187.21', '2019-04-01 12:45:12')
('Dow', '26,187.21', '2019-04-01 12:45:35')
('Dow', '26,188.17', '2019-04-01 12:45:57')
('Dow', '26,188.74', '2019-04-01 12:46:18')
('Dow', '26,200.60', '2019-04-01 13:04:31')
('Dow', '26,200.98', '2019-04-01 13:04:52')
('Dow', '26,198.66', '2019-04-01 13:05:14')
('Dow', '26,191.43', '2019-04-01 13:05:36')
('Dow', '26,191.81', '2019-04-01 13:05:59')


### SELECT DISTINCT

#### Selects on the distinct values form the "stock_name" column in the "stocks_prices" table


In [92]:
# for simplicity just the statement 
conn = sqlite3.connect("yahoostocks.db")
c = conn.cursor()

c.execute('''SELECT DISTINCT stock_name 
            FROM stocks_prices''')
for row in c:
    print(row)

('Nasdaq',)
('Crude',)
('Dow',)
('Gold',)
('S&P',)
('Appl',)
('new_stock',)


### SQL And & OR Operators

In [69]:
stock = "Dow"
date = "2019-04-01 12:44:50"

query = "SELECT * FROM stocks_prices WHERE stock_name = ?  AND time1 =?"
c.execute(query, (stock, date))
print(c.fetchone())

('Dow', '26,183.44', '2019-04-01 12:44:50')


In [70]:
stock = "Dow"
date1 = "2019-04-01 12:44:50"
date2 = "2019-04-01 12:45:12"


query = "SELECT * FROM stocks_prices WHERE stock_name = ?  AND (time1 =? OR time1 =?)"
c.execute(query,(stock, date1, date2))
print(c.fetchmany(2))

[('Dow', '26,183.44', '2019-04-01 12:44:50'), ('Dow', '26,187.21', '2019-04-01 12:45:12')]


### SQL ORDER BY keyword

In [73]:
query = "SELECT * FROM stocks_prices ORDER BY time1, stock_name DESC"
c.execute(query)
print(c.fetchmany(5))

[('Appl', 555.55, None), ('Appl', 555.55, None), ('S&P', '2,859.23', '2019-04-01 12:44:49'), ('Nasdaq', '7,804.60', '2019-04-01 12:44:50'), ('Dow', '26,183.44', '2019-04-01 12:44:50')]


### SQL LIKE Operater

In [77]:
# The lIKE operater will find pattern like a regex 
# the % sign is used to define wildcards(missing letter) both before and after the pattern

query = "SELECT * FROM stocks_prices WHERE stock_name LIKE '%o%'"
c.execute(query)
print(c.fetchmany(5))

[('Dow', '26,183.44', '2019-04-01 12:44:50'), ('Gold', '1,294.70', '2019-04-01 12:44:51'), ('Gold', '1,294.90', '2019-04-01 12:45:12'), ('Dow', '26,187.21', '2019-04-01 12:45:12'), ('Dow', '26,187.21', '2019-04-01 12:45:35')]


In [78]:

query = "SELECT * FROM stocks_prices WHERE stock_name NOT LIKE '%o%'"
c.execute(query)
print(c.fetchmany(5))

[('Nasdaq', '7,804.60', '2019-04-01 12:44:50'), ('Crude', 61.21, '2019-04-01 12:44:50'), ('S&P', '2,859.23', '2019-04-01 12:44:49'), ('Crude', 61.2, '2019-04-01 12:45:12'), ('Nasdaq', '7,804.60', '2019-04-01 12:45:12')]


### SQL Wildcards


<table>
    <tr>
        <th>Wildcard</th>
        <th>Discription</th>
    </tr>
    <tr>
        <td>%</td>
        <td>A substitute for zero or more characters</td>
    </tr>
    <tr>
        <td>_</td>
        <td>A substitute for a single character</td>
    </tr>
    <tr>
        <td>[charlist]</td>
        <td>Sets and ranges of characters to match</td>
    </tr>
    <tr>
        <td>[^charlist]</td>
        <td>Matches only a character NOT specified within the brackets</td>
    </tr>
</table>
        

### The IN Operator

In [93]:
#shorthand for OR
query= "SELECT  stock_name, price FROM stocks_prices WHERE stock_name IN ('Appl', 'Dow')"
c.execute(query)
print(c.fetchmany(5))

[('Dow', '26,183.44'), ('Dow', '26,187.21'), ('Dow', '26,187.21'), ('Dow', '26,188.17'), ('Dow', '26,188.74')]


### SQL BETWEEN Operator
#### Be careful with BETWEEN operator can produce different results in different databases

In [94]:
# the BETWEEN operator selects values with in a range. The values can be numbers, text, or dates
query = "SELECT * FROM stocks_prices WHERE time1 BETWEEN '2019-04-01 12:43:12' AND '2019-04-01 12:45:12'"
c.execute(query)
print(c.fetchall())

[('Nasdaq', '7,804.60', '2019-04-01 12:44:50'), ('Crude', 61.21, '2019-04-01 12:44:50'), ('Dow', '26,183.44', '2019-04-01 12:44:50'), ('Gold', '1,294.70', '2019-04-01 12:44:51'), ('S&P', '2,859.23', '2019-04-01 12:44:49'), ('Gold', '1,294.90', '2019-04-01 12:45:12'), ('Crude', 61.2, '2019-04-01 12:45:12'), ('Nasdaq', '7,804.60', '2019-04-01 12:45:12'), ('Dow', '26,187.21', '2019-04-01 12:45:12')]


#### NOT BETWEEN

In [100]:
query = "SELECT stock_name FROM stocks_prices WHERE stock_name NOT BETWEEN 'A' AND 'H'"
c.execute(query)
print(c.fetchmany(5))

[('Nasdaq',), ('S&P',), ('Nasdaq',), ('S&P',), ('S&P',)]


#### BETWEEN and IN

In [102]:
query = 'SELECT * FROM stocks_prices WHERE (price BETWEEN 50 AND 5000) AND NOT stock_name IN ("Gold", "Appl")'
c.execute(query)
print(c.fetchmany(5))

[('Crude', 61.21, '2019-04-01 12:44:50'), ('Crude', 61.2, '2019-04-01 12:45:12'), ('Crude', 61.18, '2019-04-01 12:45:34'), ('Crude', 61.18, '2019-04-01 12:45:57'), ('Crude', 61.18, '2019-04-01 12:46:19')]


### Alias

## SQL UPDATE Statement
### WARNING OMITTING THE WHERE CLAUSE WILL UPDATE ALL RECORDS

In [74]:
update_stock = "Appl"
update_price =555.55

query = "UPDATE stocks_prices SET price = ? WHERE stock_name = ?"
c.execute(query,(update_price, update_stock))
c.execute("SELECT * FROM stocks_prices WHERE stock_name = ?", (update_stock,))
print(c.fetchmany())

[('Appl', 555.55, None)]


## DELETE Statement
### WARNING OMITTING THE WHERE CLAUSE WILL DELETE ALL RECORDS

In [104]:
record = "new_stock"
query = "DELETE FROM stocks_prices WHERE stock_name = ?"
c.execute(query, (record,))
c.execute("SELECT * FROM stocks_prices WHERE stock_name = ?",(record,))
print(c.fetchone())

None
